#**Super-Bowl Bildanalyse**

---


# 1. Set-Up

In [42]:
# import the neccessary libraries
import os, json, cv2, random
import numpy as np
# from google.colab import drive
import matplotlib.pyplot as plt
import pandas as pd
from openpyxl.styles import Font
from tqdm import tqdm
import time
import re

In [43]:
# # OPTIONAL
# from google.colab import drive
# drive.mount('/content/drive')

In [44]:
# Load super-categories DataFrame
# NOTE: We couldn't find the super-categories in the COCO dataset and therefore we wrote them by hand in an excel file
super_categories_df = pd.read_excel(f'{os.getenv("BILDANALYSE_MODELS_COCO_DIR")}/super-categories.xlsx')
super_categories_df.set_index("class_name", inplace=True)

# 2. Installation of the pre-trained models

## 2.1 detectron2 (Object Detection)

In [45]:
#  https://detectron2.readthedocs.io/tutorials/install.html
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))


fatal: destination path 'detectron2' already exists and is not an empty directory.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment


In [46]:
# import detectron2 and its utilities
import torch
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [47]:
# Get the Configurations
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cpu"
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

[DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...
[Checkpointer] Loading from /home/arkastor/.torch/iopath_cache/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...
Reading a file from 'Detectron2 Model Zoo'


## 2.2 DEX: Deep EXpectation of apparent age from a single image (Age and Gender)

In [48]:
# !git clone 'https://github.com/serengil/tensorflow-101.git'
# NOTE: The files imported in this part were taken from the links below
#model structure: https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/age.prototxt
#pre-trained weights: https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/dex_chalearn_iccv2015.caffemodel
#age_model = cv2.dnn.readNetFromCaffe("age.prototxt", "dex_chalearn_iccv2015.caffemodel")

In [49]:

age_prototxt = f'{os.getenv("BILDANALYSE_MODELS_DEX_DIR")}/age.prototxt'
dex_model = f'{os.getenv("BILDANALYSE_MODELS_DEX_DIR")}/dex_chalearn_iccv2015.caffemodel'
gender_prototxt = f'{os.getenv("BILDANALYSE_MODELS_DEX_DIR")}/gender.prototxt'
gender_model =  f'{os.getenv("BILDANALYSE_MODELS_DEX_DIR")}/gender.caffemodel'

age_model = cv2.dnn.readNet(age_prototxt, dex_model)
gender_model = cv2.dnn.readNet(gender_prototxt, gender_model)

output_indexes = np.array([i for i in range(0, 101)]) # Set up age output range

In [50]:
#Haar cascade for face detection
opencv_home = cv2.__file__
folders = opencv_home.split(os.path.sep)[0:-1]
path = folders[0]
for folder in folders[1:]:
    path = path + "/" + folder
face_detector_path = path+"/data/haarcascade_frontalface_default.xml"
if os.path.isfile(face_detector_path) != True:
    raise ValueError("Confirm that opencv is installed on your environment! Expected path ",face_detector_path," violated.")
haar_detector = cv2.CascadeClassifier(face_detector_path)

## 2.3 FER: Facial expression recognition (Emotion)

In [51]:
!pip install fer

In [52]:
from fer import FER

## 2.4 DeepFace (Ethnicity)

'Deepface is a hybrid face recognition package. It currently wraps many state-of-the-art face recognition models: VGG-Face , Google FaceNet, OpenFace, Facebook DeepFace, DeepID, ArcFace, Dlib and SFace. The default configuration uses VGG-Face model.'
- https://github.com/serengil/deepface, accessed Nov. 16th, 2023

In [53]:
!pip install deepface

In [54]:
from deepface import DeepFace

# 3. Model Functions

## 3.1 Frame Extraction

In [55]:
def frame_extraction():
    # inspired by https://stackoverflow.com/questions/63666638/convert-video-to-frames-in-python-1-fps, accessed Oct. 26th, 2023
    KPS = fpsextractor['kps'] # Target Keyframes Per Second
    VIDEO_PATH = os.path.join(dirpath, dirname, filename) # path to current video
    FPS_OUPUT= fpsextractor['output']
    YEAR = dirname.replace("ADs_IG_", "")
    YEAR_OUTPUT_DIR = FPS_OUPUT+ "/" + YEAR
    OUTPUT_PATH = YEAR_OUTPUT_DIR + "/" + dirname + "/"
    EXTENSION = "." + fpsextractor['extension'] # file extension of exported images
    fileNameOfVideoWithoutExtension = filename[:-len("." + fpsextractor['extension'])];
    # print(OUTPUT_PATH) # e.g., ./outputs/fps_extractor/ADs_IG_2018/
    #print(OUTPUT_PATH + fileNameOfVideoWithoutExtension) # e.g., ./outputs/fps_extractor/ADs_IG_2018/AD0576

    # Ordner erstellen, in welchem je Video die Frames gepseichert werden
    # Ordner mit Jahreszahl
    if not os.path.exists(FPS_OUPUT):
        os.mkdir(FPS_OUPUT)
    if not os.path.exists(YEAR_OUTPUT_DIR):
        os.mkdir(YEAR_OUTPUT_DIR)        
    if not os.path.exists(OUTPUT_PATH):
        os.mkdir(OUTPUT_PATH)
    # Unterordner mit Video-ID
    if not os.path.exists(OUTPUT_PATH + fileNameOfVideoWithoutExtension):
        os.mkdir(OUTPUT_PATH + fileNameOfVideoWithoutExtension)

    # print(KPS, IMAGE_PATH, EXTENSION)

    cap = cv2.VideoCapture(VIDEO_PATH)
    fps = round(cap.get(cv2.CAP_PROP_FPS))
    print(fps)
    # exit()
    hop = round(fps / KPS)
    curr_frame = 0
    while(True):
        ret, frame = cap.read()
        if not ret: break
        if curr_frame % hop == 0:
            name = OUTPUT_PATH + fileNameOfVideoWithoutExtension + "/" + fileNameOfVideoWithoutExtension + "_Frame_" + str(curr_frame) + EXTENSION
            # print(name)
            cv2.imwrite(name, frame)
        curr_frame += 1
    cap.release()

## 3.2 Predictions


### 3.2.1 detectron2

In [56]:
# Functions for finding the quadrant_number
def create_nine_quadrants(image):
    height, width = image.shape[:2]
    quadrant_width = width // 3
    quadrant_height = height // 3

    quadrants = {}

    for i in range(3):
        for j in range(3):
            quadrant_num = i * 3 + j + 1
            x1 = j * quadrant_width
            y1 = i * quadrant_height
            x2 = x1 + quadrant_width
            y2 = y1 + quadrant_height
            quadrants[quadrant_num] = [(x1, y1), (x2, y2)]

    def get_quadrant_for_coordinate(coord):
        x, y = coord
        for quadrant_num, ((x1, y1), (x2, y2)) in quadrants.items():
            if x1 <= x < x2 and y1 <= y < y2:
                return quadrant_num
        return None

    return get_quadrant_for_coordinate

In [57]:
# Function for creating cropped image
def crop_human_bounding_box(image, bounding_box):
    # Get coordinates of the bounding box
    x_min, y_min, x_max, y_max = map(int, bounding_box)

    # Ensure coordinates are within the image boundaries
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    x_max = min(image.shape[1], x_max)
    y_max = min(image.shape[0], y_max)

    # Crop the human region from the image based on the bounding box
    cropped_human = image[y_min:y_max, x_min:x_max]

    return cropped_human

In [58]:
# Function for visualising outputs
def detectron2_visualisation(im, outputs):
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    #Change image to RGB
    image_bgr = out.get_image()
    image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Generate output path for the annotated image
    img_filename = os.path.basename(image_path)
    output_filename = img_filename.split('.')[0] + f'_detectron_annotated.png'
    output_path = os.path.join(visualiser_folder_path_ad, output_filename)

    cv2.imwrite(output_path, image)

In [59]:
def detectron2_analysis(im):
    outputs = predictor(im)
    metadata = MetadataCatalog.get(cfg.DATASETS.TEST[0])
    class_names = metadata.get("thing_classes", None)

    instances = outputs["instances"].to("cpu")

    prediction_objects = []

    # Iterate over objects found
    for i in range(len(instances)):
        bbox = instances.pred_boxes.tensor.cpu().numpy()[i].squeeze()

        # Area Calculation
        x1, y1, x2, y2 = bbox
        area_object = (x2 - x1) * (y2 - y1)
        height, width = im.shape[:2]
        area_image = height * width
        object_propotion = area_object/ area_image

        # Center-Point Calculation
        center_point = [(x1 + x2) / 2, (y1 + y2) / 2]
        get_quadrant_for_coordinate = create_nine_quadrants(im)
        quadrant_number = get_quadrant_for_coordinate(center_point)

        # Cropping image and predicting the other attributes
        if instances.pred_classes[i].item() == 0:
           human_image = crop_human_bounding_box(im, bbox)
           age_prediction, age_group_prediction, gender_prediction = dex_analysis(human_image.copy(), visualiser, i)
           emotion_prediction = fer_analysis(human_image.copy(), visualiser, i)
           ethnicity_prediction = deepface_analysis(human_image.copy(), visualiser, i)
        else:
            age_prediction = age_group_prediction = gender_prediction = emotion_prediction = ethnicity_prediction = "-"

        # Saving prediction data
        data = {
            "video_frame": element,
            "class_id": instances.pred_classes[i].item(),
            "class_name": class_names[instances.pred_classes[i].item()],
            "confidence": instances.scores[i].item(),
            "object_propotion": object_propotion,
            "quadrant_number": quadrant_number,
            "age_prediction": age_prediction,
            "age_group_prediction": age_group_prediction,
            "gender_prediction": gender_prediction,
            "emotion_prediction": emotion_prediction,
            "ethnicity_prediction": ethnicity_prediction
        }
        prediction_objects.append(data)

    if visualiser:
      detectron2_visualisation(im, outputs)

    return prediction_objects

### 3.2.2 DEX: Deep EXpectation of apparent age from a single image (Age and Gender)

In [60]:
# Detecting faces
def detect_faces(img):

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_unsorted = haar_detector.detectMultiScale(gray, 1.3, 5)

    # Sort the faces so that only the biggest bounding box is detected
    faces = sorted(faces_unsorted, key=lambda face: face[2] * face[3], reverse=True)

    return faces

In [61]:
# Visualising output
def dex_visualisation(img, face, age, gender, i):
    # Bounding Box
    x, y, w, h = face
    # Draw rectangle around detected face
    cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)

    # Annotate with age and gender information
    text = f'Age: {age}, Gender: {gender}'
    cv2.putText(img, text, (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Generate output path for the annotated image
    img_filename = os.path.basename(image_path)
    output_filename = img_filename.split('.')[0] + f'_age_gender_{i}_annotated.png'
    output_path = os.path.join(visualiser_folder_path_ad, output_filename)

    # Save the annotated image with bounding boxes and emotions
    cv2.imwrite(output_path, img)

In [62]:
def dex_analysis(img, visualiser, i):
    # Detect faces in the image
    faces = detect_faces(img)

    if faces is not None and len(faces) > 0:
        face = faces[0]
        x, y, w, h = face

        detected_face = img[int(y):int(y+h), int(x):int(x+w)]
        detected_face = cv2.resize(detected_face, (224, 224))
        img_blob = cv2.dnn.blobFromImage(detected_face)

        # Age prediction
        age_model.setInput(img_blob)
        age_dist = age_model.forward()[0]
        age = round(np.sum(age_dist * output_indexes))

        # Age Group (based on a similar study by Chaudhari, S. J., & Kagalkar, R. M. (2015). Methodology for gender identification, classification and recognition of human age. International Journal of Computer Applications, 975, 8887.)
        age_group = [0, 8] if 0 < age <= 8 else \
                    [9, 17] if 9 <= age < 18 else \
                    [18, 30] if 18 <= age < 31 else \
                    [31, 60] if 31 <= age < 61 else \
                    [61, 100] if 61 <= age < 100 else \
                    [100, 200]

        # Gender prediction
        gender_model.setInput(img_blob)
        gender_class = gender_model.forward()[0]
        gender = 'Woman' if np.argmax(gender_class) == 0 else 'Man'

        if visualiser:
            dex_visualisation(img, face, age, gender, i)
    else:
        age = age_group = gender = "-"

    return age, age_group, gender


### 3.2.3 FER: Facial expression recognition

In [63]:
def fer_visualisation(img, result, emotions, i):
    # Get face bounding box coordinates
    x, y, w, h = result['box']

    # Draw a bounding box around the detected face
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Get the dominant emotion and its probability for the face
    dominant_emotion = max(emotions, key=emotions.get)
    emotion_probability = emotions[dominant_emotion]

    # Annotate the image with the dominant emotion for each face
    cv2.putText(img, f'{dominant_emotion} ({emotion_probability:.2f})',(x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

    # Generate output path for the annotated image
    img_filename = os.path.basename(image_path)
    output_filename = img_filename.split('.')[0] + f'_emotion_{i}_annotated.png'
    output_path = os.path.join(visualiser_folder_path_ad, output_filename)

    # Save the annotated image with bounding boxes and emotions
    cv2.imwrite(output_path, img)


In [64]:
def fer_analysis(img, visualiser, i):
    # Initialize the FER model
    detector = FER(mtcnn=True)

    try:
      # Detect faces and their emotions in the image
      results = detector.detect_emotions(img)

      # Sort the results so that only the biggest bounding box is detected
      sorted_results = sorted(results, key=lambda x: (x['box'][2] * x['box'][3]), reverse=True)
      result= sorted_results[0]

      # Get detected emotions for the face
      emotions = result['emotions']

      # Get the dominant emotion and its probability for the face
      dominant_emotion = max(emotions, key=emotions.get)

      if visualiser:
          fer_visualisation(img, result, emotions, i)

    except:
        dominant_emotion = "-"

    return dominant_emotion

### 3.2.4 DeepFace

In [65]:
def deepface_visualiser(img, prediction, i):

    x, y, w, h = prediction['region']['x'], prediction['region']['y'], prediction['region']['w'], prediction['region']['h']

    #draw the rectangle
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    #plot text on image
    cv2.putText(img, str(prediction['dominant_race']), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Generate output path for the annotated image
    img_filename = os.path.basename(image_path)
    output_filename = img_filename.split('.')[0] + f'_ethnicity_{i}_annotated.png'
    output_path = os.path.join(visualiser_folder_path_ad, output_filename)

    # Save the annotated image with bounding boxes and emotions
    cv2.imwrite(output_path, img)

In [66]:
def deepface_analysis(img, visualiser, i):

    try:
      predictions = DeepFace.analyze(img, actions = ['race'])

      sorted_predictions = sorted(predictions, key=lambda x: x['region']['w'] * x['region']['h'], reverse=True)
      prediction = sorted_predictions[0]
      dominant_ethnicity = prediction['dominant_race']

      if visualiser:
          deepface_visualiser(img, prediction, i)

    except:
      dominant_ethnicity = "-"

    return dominant_ethnicity


## 3.2 Summary

### 3.2.1 Summary (1)

In [67]:
def generate_summary(input_file_path):
    # Read the Excel file
    data = pd.read_excel(input_file_path, sheet_name='Predictions', header=[0])

    # Filter out the instances where objects are smaller than the schwellenwert_proportion_detectron_2
    filtered_predictions_classes = data[data['object_propotion'] >= schwellenwert_proportion_detectron_2]

    # Calculate the summary for classes
    summary_classes_sheet = filtered_predictions_classes.groupby('class_name').size().reset_index(name='count')
    summary_classes_sheet['avg_confidence'] = filtered_predictions_classes.groupby('class_name')['confidence'].mean().values.round(4)
    summary_classes_sheet['avg_object_propotion'] = filtered_predictions_classes.groupby('class_name')['object_propotion'].mean().values.round(4)
    summary_classes_sheet['avg_quadrant_number'] = filtered_predictions_classes.groupby('class_name')['quadrant_number'].mean().values.round()
    summary_classes_sheet['frame_nr'] = filtered_predictions_classes.groupby('class_name')['video_frame'].nunique().reset_index()['video_frame']

    # Calculate the frame_nr_ratio
    total_frame_number = filtered_predictions_classes.iloc[0, 13]
    summary_classes_sheet['frame_ratio'] = round(summary_classes_sheet['frame_nr'] / total_frame_number, 4)

    # Filter rows based on the frame number
    summary_classes_sheet = summary_classes_sheet[summary_classes_sheet['frame_nr'] >= schwellenwert_frame_nr_detectron_2]

    # Sort by count
    summary_classes_sheet = summary_classes_sheet.sort_values(by='count', ascending=False)

    # Define columns to compute counts for in the original data
    columns_to_count = ['super-category', 'gender_prediction', 'ethnicity_prediction', 'age_group_prediction', 'emotion_prediction']

    data.columns = [re.sub(r'Unnamed:\s*\d*', '', col) if 'Unnamed' in str(col) else col for col in data.columns]

    # Write to Excel
    with pd.ExcelWriter(input_file_path, engine='openpyxl') as writer:
        # Saving the excel files
        data.to_excel(writer, sheet_name='Predictions', index=False, index_label=None)
        summary_classes_sheet.to_excel(writer, sheet_name='Summary_Objects', index=False)

        # Update the summary attributes sheet
        for column in columns_to_count:

            # Filter out the empty instances
            non_empty_predictions = data[data[column] != "-"]

            # Filter out the instances where objects are smaller than the schwellenwert_proportion_detectron_2 or schwellenwert_proportion_human_attributes
            if column == 'super-category':
              filtered_predictions_human = non_empty_predictions[non_empty_predictions['object_propotion'] >= schwellenwert_proportion_detectron_2]
            else:
              filtered_predictions_human = non_empty_predictions[non_empty_predictions['object_propotion'] >= schwellenwert_proportion_human_attributes]

            # Calculate the summary for human attributes
            summary_attributes_sheet = filtered_predictions_human.groupby(column).size().reset_index(name='count_' + column)
            summary_attributes_sheet[f'avg_object_propotion_{column}'] = filtered_predictions_human.groupby(column)['object_propotion'].mean().values.round(4)
            summary_attributes_sheet[f'avg_quadrant_number_{column}'] = filtered_predictions_human.groupby(column)['quadrant_number'].mean().values.round()
            summary_attributes_sheet[f'frame_nr_{column}'] = filtered_predictions_human.groupby(column)['video_frame'].nunique().reset_index()['video_frame']
            summary_attributes_sheet[f'frame_ratio_{column}'] = round(summary_attributes_sheet[f'frame_nr_{column}'] / total_frame_number, 4)

            # Filter out the instances where the number of unique frames are smaller than the schwellenwert_frame_nr_detectron_2 or schwellenwert_frame_nr_human_attributes
            if column == "super-category":
              summary_attributes_sheet = summary_attributes_sheet[summary_attributes_sheet[f'frame_nr_{column}'] >= schwellenwert_frame_nr_detectron_2]
            else:
              summary_attributes_sheet = summary_attributes_sheet[summary_attributes_sheet[f'frame_nr_{column}'] >= schwellenwert_frame_nr_human_attributes]

            summary_attributes_sheet.sort_values(by='count_' + column, ascending=False, inplace=True)
            summary_attributes_sheet.to_excel(writer, sheet_name='Summary_Objects', startrow=0, startcol=(columns_to_count.index(column) * 7) + 8, index=False)

### 3.2.2 Summary Gender & Ethnicity


In [68]:
def generate_summary_gender_ethnicity(input_file_path):

    data = pd.read_excel(input_file_path, sheet_name = 'Predictions')
    summary_1 = pd.read_excel(input_file_path, sheet_name='Summary')

    ########################################################################### Gender ##############################################################################################
    data_gender = data[data['gender_prediction'] != '-']

    # Filter out the instances where objects are smaller than the schwellenwert_proportion_human_attributes
    data_gender = data_gender[data_gender['object_propotion'] >= schwellenwert_proportion_human_attributes]

    # Filter out the instances where the number of unique frames are smaller than the schwellenwert_frame_nr_human_attributes
    unique_genders = data_gender['gender_prediction'].unique()
    genders_to_remove = []

    for gender in unique_genders:
        filtered_df = data_gender[data_gender['gender_prediction'] == gender]
        unique_video_frames = filtered_df['video_frame'].unique().tolist()

        if len(unique_video_frames) < schwellenwert_frame_nr_human_attributes:
            genders_to_remove.append(gender)

    data_gender = data_gender[~data_gender['gender_prediction'].isin(genders_to_remove)]

    #New sheet
    Gender_sheet = pd.DataFrame(columns=['case', 'count', 'frame_ratio',
                                        'avg_object_propotion_women', 'avg_quadrant_number_women',
                                        'quadrant_numbers_women',
                                        'avg_object_propotion_men', 'avg_quadrant_number_men',
                                        'quadrant_numbers_men'
                                        ])
    case_data_gender = {}

    # Iterate through the data to determine cases and calculate averages
    for frame_key, frame_data in data_gender.groupby('video_frame'):
        # Count the occurrences of each gender in the frame
        gender_counts = frame_data['gender_prediction'].value_counts()

        # Create the case string dynamically based on the gender counts
        case = ' & '.join([f'{count} {gender}' for gender, count in sorted(gender_counts.items())])

        # Check if the case already exists in the case_data dictionary
        if case not in case_data_gender:
            case_data_gender[case] = {'count': 0,
                              'total_object_propotion_women': 0, 'total_quadrant_number_women': 0,
                              'quadrant_numbers_women': [],
                              'total_object_propotion_men': 0, 'total_quadrant_number_men': 0,
                              'quadrant_numbers_men': []
                              }

        # Update cumulative values for the case
        case_data_gender[case]['count'] += 1
        case_data_gender[case]['total_object_propotion_women'] += frame_data[frame_data['gender_prediction'] == 'Woman']['object_propotion'].mean()
        case_data_gender[case]['total_quadrant_number_women'] += frame_data[frame_data['gender_prediction'] == 'Woman']['quadrant_number'].mean()
        case_data_gender[case]['quadrant_numbers_women'].extend(frame_data[frame_data['gender_prediction'] == 'Woman']['quadrant_number'].tolist())

        case_data_gender[case]['total_object_propotion_men'] += frame_data[frame_data['gender_prediction'] == 'Man']['object_propotion'].mean()
        case_data_gender[case]['total_quadrant_number_men'] += frame_data[frame_data['gender_prediction'] == 'Man']['quadrant_number'].mean()
        case_data_gender[case]['quadrant_numbers_men'].extend(frame_data[frame_data['gender_prediction'] == 'Man']['quadrant_number'].tolist())

    # Calculate averages for each case
    for case, values in case_data_gender.items():
        count = values['count']

        # Calculate averages for women
        avg_object_propotion_women = round(values['total_object_propotion_women'] / count, 4) if count > 0 else 0
        avg_quadrant_number_women = round(values['total_quadrant_number_women'] / count) if count > 0 and not np.isnan(values['total_quadrant_number_women']) else 0

        # Calculate averages for men
        avg_object_propotion_men = round(values['total_object_propotion_men'] / count, 4) if count > 0 else 0
        avg_quadrant_number_men = round(values['total_quadrant_number_men'] / count) if count > 0 and not np.isnan(values['total_quadrant_number_men']) else 0

        # Calculate frame ratio
        total_frame_number = data.iloc[0, 13]
        frame_ratio = round(count/ total_frame_number, 4)

        # Append the values to the Gender_sheet
        Gender_sheet = Gender_sheet.append({
            'case': case,
            'count': count,
            'frame_ratio': frame_ratio,
            'avg_object_propotion_women': avg_object_propotion_women,
            'avg_quadrant_number_women': avg_quadrant_number_women,
            'quadrant_numbers_women': values['quadrant_numbers_women'],
            'avg_object_propotion_men': avg_object_propotion_men,
            'avg_quadrant_number_men': avg_quadrant_number_men,
            'quadrant_numbers_men': values['quadrant_numbers_men']
        }, ignore_index=True)

    Gender_sheet = Gender_sheet.sort_values(by='count', ascending=False)

    # Replace empty spaces with 0 in the entire dataframe
    Gender_sheet[['avg_object_propotion_women', 'avg_object_propotion_men']] = Gender_sheet[['avg_object_propotion_women', 'avg_object_propotion_men']].fillna(0)


    ########################################################################### Ethnicity #########################################################################################
    data_ethnicity = data[data['ethnicity_prediction'] != '-']

    # Filter out the instances where objects are smaller than the schwellenwert_proportion_human_attributes
    data_ethnicity = data_ethnicity[data_ethnicity['object_propotion'] >= schwellenwert_proportion_human_attributes]

    # Filter out the instances where the number of unique frames are smaller than the schwellenwert_frame_nr_human_attributes
    unique_ethnicities = data_ethnicity['ethnicity_prediction'].unique()
    ethnicity_to_remove = []

    for ethnicity in unique_ethnicities:
        filtered_df = data_ethnicity[data_ethnicity['ethnicity_prediction'] == ethnicity]
        unique_video_frames = filtered_df['video_frame'].unique().tolist()

        if len(unique_video_frames) < schwellenwert_frame_nr_human_attributes:
            ethnicity_to_remove.append(ethnicity)

    data_ethnicity = data_ethnicity[~data_ethnicity['ethnicity_prediction'].isin(ethnicity_to_remove)]

    # New sheet
    Ethnicity_sheet = pd.DataFrame(columns=['case', 'count', 'frame_ratio',
                                            'avg_object_propotion_asian', 'avg_quadrant_number_asian',
                                            'quadrant_numbers_asian',
                                            'avg_object_propotion_black', 'avg_quadrant_number_black',
                                            'quadrant_numbers_black',
                                            'avg_object_propotion_indian', 'avg_quadrant_number_indian',
                                            'quadrant_numbers_indian',
                                            'avg_object_propotion_latino_hispanic', 'avg_quadrant_number_latino_hispanic',
                                            'quadrant_numbers_latino_hispanic',
                                            'avg_object_propotion_middle_eastern', 'avg_quadrant_number_middle_eastern',
                                            'quadrant_numbers_middle_eastern',
                                            'avg_object_propotion_white', 'avg_quadrant_number_white',
                                            'quadrant_numbers_white'
                                            ])

    case_data_ethnicity = {}

    # Iterate through the data to determine cases and calculate averages
    for frame_key, frame_data in data_ethnicity.groupby('video_frame'):
        # Count the occurrences of each ethnicity in the frame
        ethnicity_counts = frame_data['ethnicity_prediction'].value_counts()

        # Create the case string dynamically based on the ethnicity counts
        case = ' & '.join([f'{count} {ethnicity}' for ethnicity, count in sorted(ethnicity_counts.items())])

        # Check if the case already exists in the case_data dictionary
        if case not in case_data_ethnicity:
            case_data_ethnicity[case] = {'count': 0,
                              'total_object_propotion_asian': 0, 'total_quadrant_number_asian': 0,
                              'quadrant_numbers_asian': [],
                              'total_object_propotion_black': 0, 'total_quadrant_number_black': 0,
                              'quadrant_numbers_black': [],
                              'total_object_propotion_indian': 0, 'total_quadrant_number_indian': 0,
                              'quadrant_numbers_indian': [],
                              'total_object_propotion_latino_hispanic': 0, 'total_quadrant_number_latino_hispanic': 0,
                              'quadrant_numbers_latino_hispanic': [],
                              'total_object_propotion_middle_eastern': 0, 'total_quadrant_number_middle_eastern': 0,
                              'quadrant_numbers_middle_eastern': [],
                              'total_object_propotion_white': 0, 'total_quadrant_number_white': 0,
                              'quadrant_numbers_white': []
                              }

        # Update cumulative values for the case
        case_data_ethnicity[case]['count'] += 1
        case_data_ethnicity[case]['total_object_propotion_asian'] += frame_data[frame_data['ethnicity_prediction'] == 'asian']['object_propotion'].mean()
        case_data_ethnicity[case]['total_quadrant_number_asian'] += frame_data[frame_data['ethnicity_prediction'] == 'asian']['quadrant_number'].mean()
        case_data_ethnicity[case]['quadrant_numbers_asian'].extend(frame_data[frame_data['ethnicity_prediction'] == 'asian']['quadrant_number'].tolist())

        case_data_ethnicity[case]['total_object_propotion_black'] += frame_data[frame_data['ethnicity_prediction'] == 'black']['object_propotion'].mean()
        case_data_ethnicity[case]['total_quadrant_number_black'] += frame_data[frame_data['ethnicity_prediction'] == 'black']['quadrant_number'].mean()
        case_data_ethnicity[case]['quadrant_numbers_black'].extend(frame_data[frame_data['ethnicity_prediction'] == 'black']['quadrant_number'].tolist())

        case_data_ethnicity[case]['total_object_propotion_indian'] += frame_data[frame_data['ethnicity_prediction'] == 'indian']['object_propotion'].mean()
        case_data_ethnicity[case]['total_quadrant_number_indian'] += frame_data[frame_data['ethnicity_prediction'] == 'indian']['quadrant_number'].mean()
        case_data_ethnicity[case]['quadrant_numbers_indian'].extend(frame_data[frame_data['ethnicity_prediction'] == 'indian']['quadrant_number'].tolist())

        case_data_ethnicity[case]['total_object_propotion_latino_hispanic'] += frame_data[frame_data['ethnicity_prediction'] == 'latino hispanic']['object_propotion'].mean()
        case_data_ethnicity[case]['total_quadrant_number_latino_hispanic'] += frame_data[frame_data['ethnicity_prediction'] == 'latino hispanic']['quadrant_number'].mean()
        case_data_ethnicity[case]['quadrant_numbers_latino_hispanic'].extend(frame_data[frame_data['ethnicity_prediction'] == 'latino hispanic']['quadrant_number'].tolist())

        case_data_ethnicity[case]['total_object_propotion_middle_eastern'] += frame_data[frame_data['ethnicity_prediction'] == 'middle eastern']['object_propotion'].mean()
        case_data_ethnicity[case]['total_quadrant_number_middle_eastern'] += frame_data[frame_data['ethnicity_prediction'] == 'middle eastern']['quadrant_number'].mean()
        case_data_ethnicity[case]['quadrant_numbers_middle_eastern'].extend(frame_data[frame_data['ethnicity_prediction'] == 'middle eastern']['quadrant_number'].tolist())

        case_data_ethnicity[case]['total_object_propotion_white'] += frame_data[frame_data['ethnicity_prediction'] == 'white']['object_propotion'].mean()
        case_data_ethnicity[case]['total_quadrant_number_white'] += frame_data[frame_data['ethnicity_prediction'] == 'white']['quadrant_number'].mean()
        case_data_ethnicity[case]['quadrant_numbers_white'].extend(frame_data[frame_data['ethnicity_prediction'] == 'white']['quadrant_number'].tolist())

    # Calculate averages for each case
    for case, values in case_data_ethnicity.items():
        count = values['count']

        # Calculate averages for asian
        avg_object_propotion_asian = round(values['total_object_propotion_asian'] / count, 4) if count > 0 else 0
        avg_quadrant_number_asian = round(values['total_quadrant_number_asian'] / count) if count > 0 and not np.isnan(values['total_quadrant_number_asian']) else 0

        # Calculate averages for black
        avg_object_propotion_black = round(values['total_object_propotion_black'] / count, 4) if count > 0 else 0
        avg_quadrant_number_black = round(values['total_quadrant_number_black'] / count) if count > 0 and not np.isnan(values['total_quadrant_number_black']) else 0

        # Calculate averages for indian
        avg_object_propotion_indian = round(values['total_object_propotion_indian'] / count, 4) if count > 0 else 0
        avg_quadrant_number_indian = round(values['total_quadrant_number_indian'] / count) if count > 0 and not np.isnan(values['total_quadrant_number_indian']) else 0

        # Calculate averages for latino_hispanic
        avg_object_propotion_latino_hispanic = round(values['total_object_propotion_latino_hispanic'] / count, 4) if count > 0 else 0
        avg_quadrant_number_latino_hispanic = round(values['total_quadrant_number_latino_hispanic'] / count) if count > 0 and not np.isnan(values['total_quadrant_number_latino_hispanic']) else 0


        # Calculate averages for middle_eastern
        avg_object_propotion_middle_eastern = round(values['total_object_propotion_middle_eastern'] / count, 4) if count > 0 else 0
        avg_quadrant_number_middle_eastern = round(values['total_quadrant_number_middle_eastern'] / count) if count > 0 and not np.isnan(values['total_quadrant_number_middle_eastern']) else 0

        # Calculate averages for white
        avg_object_propotion_white = round(values['total_object_propotion_white'] / count, 4) if count > 0 else 0
        avg_quadrant_number_white = round(values['total_quadrant_number_white'] / count) if count > 0 and not np.isnan(values['total_quadrant_number_white']) else 0

        # Calculate frame ratio
        total_frame_number = data.iloc[0, 13]
        frame_ratio = round(count/ total_frame_number, 4)

        # Append the values to the Ethnicity_sheet
        Ethnicity_sheet = Ethnicity_sheet.append({
            'case': case,
            'count': count,
            'frame_ratio': frame_ratio,
            'avg_object_propotion_asian': avg_object_propotion_asian,
            'avg_quadrant_number_asian': avg_quadrant_number_asian,
            'quadrant_numbers_asian': values['quadrant_numbers_asian'],
            'avg_object_propotion_black': avg_object_propotion_black,
            'avg_quadrant_number_black': avg_quadrant_number_black,
            'quadrant_numbers_black': values['quadrant_numbers_black'],
            'avg_object_propotion_indian': avg_object_propotion_indian,
            'avg_quadrant_number_indian': avg_quadrant_number_indian,
            'quadrant_numbers_indian': values['quadrant_numbers_indian'],
            'avg_object_propotion_latino_hispanic': avg_object_propotion_latino_hispanic,
            'avg_quadrant_number_latino_hispanic': avg_quadrant_number_latino_hispanic,
            'quadrant_numbers_latino_hispanic': values['quadrant_numbers_latino_hispanic'],
            'avg_object_propotion_middle_eastern': avg_object_propotion_middle_eastern,
            'avg_quadrant_number_middle_eastern': avg_quadrant_number_middle_eastern,
            'quadrant_numbers_middle_eastern': values['quadrant_numbers_middle_eastern'],
            'avg_object_propotion_white': avg_object_propotion_white,
            'avg_quadrant_number_white': avg_quadrant_number_white,
            'quadrant_numbers_white': values['quadrant_numbers_white'],
        }, ignore_index=True)

    # Replace empty spaces with 0 in the entire dataframe
    Ethnicity_sheet[['avg_object_propotion_asian', 'avg_object_propotion_black', 'avg_object_propotion_indian',
                    'avg_object_propotion_latino_hispanic', 'avg_object_propotion_middle_eastern', 'avg_object_propotion_white']] = \
        Ethnicity_sheet[['avg_object_propotion_asian', 'avg_object_propotion_black', 'avg_object_propotion_indian',
                        'avg_object_propotion_latino_hispanic', 'avg_object_propotion_middle_eastern', 'avg_object_propotion_white']].fillna(0)

    Ethnicity_sheet = Ethnicity_sheet.sort_values(by='count', ascending=False)

    data.columns = [re.sub(r'Unnamed:\s*\d*', '', col) if 'Unnamed' in str(col) else col for col in data.columns]
    summary_1.columns = [re.sub(r'Unnamed:\s*\d*', '', col) if 'Unnamed' in str(col) else col for col in summary_1.columns]

    with pd.ExcelWriter(input_file_path, engine='openpyxl') as writer:
        data.to_excel(writer, sheet_name='Predictions', index=False, index_label=None)
        summary_1.to_excel(writer, sheet_name='Summary_Objects', index= False, index_label=None)
        Gender_sheet.to_excel(writer, sheet_name='Summary_Gender_Ethnicity', index=False)
        Ethnicity_sheet.to_excel(writer, sheet_name='Summary_Gender_Ethnicity', startrow=0, startcol=10, index=False)

# 4. Main

## 4.1 Extracting the Frames

In [69]:
# settings
fpsextractor = {
    "format": r"mp4", # Video Format of ads within sources.ads folder.
    "yearsToEvaluate": [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022], # A list of the years which will be analysed.

    # fot OpenCV
    "kps": 3, # Target Keyframes Per Second
    "input": os.getenv("ADS_DIR"), # Path to the downloaded Ads. The folder this path belongs to must contain subfolgers like "ADs_IG_2013" to divide the videos.
    "output": os.getenv("INPUT_FRAMES_ALL"), # Output path where to store exported images
    "extension": r"png", # File Extension for the exported images
}

In [70]:
# Liste der auszuwertenden Jahre aus conf laden und zu Strings umwanzeln.
yearsAsString = map(str, fpsextractor['yearsToEvaluate'])
yearsAsString = list(yearsAsString)
yearsAsString = yearsAsString[0] # TODO Delete nce script works
print("yearsasstring",yearsAsString)
print("fpsextractorinput", fpsextractor['input'])
# Durch Werbevideos iterieren und nur die Jahre auswerten, die in conf hinterlegt sind.
for dirpath, dirnames, filenames in os.walk(fpsextractor['input']):
    for dirname in dirnames:
        print ("dirname", dirname)
        # wenn aktuell betrachteter Ordner (Jahr) in Config hinterlegt ist, dann auswerten
        if (dirname[-4:] in yearsAsString):
            print('# ', dirname[-4:], dirname) # print year and folder name

            for filename in os.listdir(os.path.join(dirpath, dirname)):
                # if filename.endswith('.' + cfg.fpsextractor['format']):
                    # print(os.path.join(dirpath, dirname, filename))
                    print(filename)
                    frame_extraction()

yearsasstring 2013
fpsextractorinput /home/arkastor/Development/Commercial-Brand-Differentiating-Message-Analysis/ADs
dirname ADs_IG_2015
dirname ADs_IG_2017
dirname ADs_IG_2020
dirname ADs_IG_2019
dirname ADs_IG_2018
dirname ADs_IG_2022
dirname ADs_IG_2013
#  2013 ADs_IG_2013
AD0254.mp4
30
AD0277.mp4
30
AD0264.mp4
30
AD0298.mp4
30
AD0269.mp4
30
AD0273.mp4
30
AD0266.mp4
30
AD0292.mp4
30
AD0290.mp4
30
AD0282.mp4
30
AD0291.mp4
30
AD0284.mp4
30
AD0293.mp4
30
AD0275.mp4
30
AD0262.mp4
30
AD0286.mp4
30
AD0278.mp4
30
AD0300.mp4
30
AD0297.mp4
30
AD0276.mp4
30
AD0267.mp4
30
AD0263.mp4
30
AD0280.mp4
30
AD0256.mp4
30
AD0272.mp4
30
AD0295.mp4
30
AD0253.mp4
30
AD0299.mp4
30
AD0270.mp4
30
AD0288.mp4
30
AD0252.mp4
30
AD0265.mp4
30
AD0296.mp4
30
AD0271.mp4
30
AD0287.mp4
30
AD0268.mp4
30
AD0285.mp4
30
AD0294.mp4
30
AD0279.mp4
30
AD0260.mp4
30
AD0289.mp4
30
AD0258.mp4
30
AD0261.mp4
30
AD0259.mp4
30
AD0255.mp4
30
AD0281.mp4
30
AD0257.mp4
30
dirname ADs_IG_2016
dirname ADs_IG_2014
dirname ADs_IG_2021


## 4.2 Creating the output files

In [71]:
# Path to the folder containing the frames and the excel lists
input_folder_path = os.getenv("INPUT_FRAMES_ALL")
# output_folder_path = '/content/drive/MyDrive/SuperBowl_Project_FUB/output_lists'
output_folder_path = os.getenv("OUTPUT_BILD_PLUS_TON_LISTS_DIR")

In [72]:
# If you want to save the output images then set visualiser to 1
visualiser = 0
if visualiser:
    visualiser_folder_path = os.path.join(output_folder_path, "visualiser")
    os.makedirs(visualiser_folder_path)

In [73]:
# OPTIONAL: So that the loops starts with the years in an alphabetical order
years = []
for year in os.listdir(input_folder_path):
  print("year detected", year)
  years.append(year)
years.sort()

year detected 2013


In [74]:
# Creating the output folders
for year in years:
    output_year_folder_path = os.path.join(output_folder_path, "ADs_IG_"+year)
    print("output_year_folder_path created", output_year_folder_path)
    os.makedirs(output_year_folder_path, exist_ok=True)

output_year_folder_path created /home/arkastor/Development/Commercial-Brand-Differentiating-Message-Analysis/Final_Files/03. Output Bild + Ton/01. output_lists/ADs_IG_2013


In [75]:
# Creating the output excel files
for year in years:
    # Save the ad folders in a list
    ad_folder_path = os.path.join(input_folder_path, year, "ADs_IG_" + year)
    print("ad_folder_path ", ad_folder_path)
    all_ADs = os.listdir(ad_folder_path)
    AD_names = [item for item in all_ADs if os.path.isdir(os.path.join(ad_folder_path, item))]
    sorted_AD_names = sorted(AD_names)

    # Run the code for each ad
    while len(sorted_AD_names) > 0:
        all_predictions = []
        all_frames = []
        current_AD = sorted_AD_names.pop(0)
        active_ad_folder_path = os.path.join(ad_folder_path, current_AD)

        if visualiser:
            visualiser_folder_path_ad = os.path.join(visualiser_folder_path, current_AD)
            os.makedirs(visualiser_folder_path_ad)

        for frame in os.listdir(active_ad_folder_path):
            if frame.endswith(".png"):
                all_frames.append(frame)

        for element in tqdm(all_frames, desc=f"Processing {current_AD}", unit="element"):
            image_path = os.path.join(active_ad_folder_path, element)
            im = cv2.imread(image_path)

            prediction_objects = detectron2_analysis(im)
            all_predictions.extend(prediction_objects)

        # Extending all_predictions with super-categories used for the setting
        # NOTE: The reason why we have used "try and except" is because for AD0347 and AD0754 Detectron2 wasn't able to detect any object since only text and a QR-Code was shown in the ad.
        all_predictions = pd.DataFrame(all_predictions)
        try:
          all_predictions_super_categories = all_predictions.join(super_categories_df, on="class_name")
        except:
          all_predictions_super_categories = all_predictions

        # Calculate total frame number
        total_frame_number = len([file for file in os.listdir(active_ad_folder_path) if file.lower().endswith('.png')])

        # To Excel
        output_year_folder_path = os.path.join(output_folder_path,"ADs_IG_"+year)
        AD_name = current_AD + ".xlsx"
        output_file = os.path.join(output_year_folder_path, AD_name)
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            all_predictions_super_categories.to_excel(writer, sheet_name='Predictions', index=False)

            # Add total frame number to the Excel sheet
            sheet = writer.sheets['Predictions']
            sheet[f'N1'] = 'Total Frame Number'
            sheet[f'N2'] = total_frame_number

ad_folder_path  /home/arkastor/Development/Commercial-Brand-Differentiating-Message-Analysis/01_input_frames_all/2013/ADs_IG_2013


Processing AD0252:   0%|          | 0/184 [00:00<?, ?element/s]

Creating converter from 3 to 5


2024-11-12 17:18:23.051450: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1731428303.051513  322717 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10289 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9
Processing AD0254:  81%|████████▏ | 70/86 [02:02<00:27,  1.74s/element]


KeyboardInterrupt: 

## 4.3 Creating the summaries

In [ ]:
# NOTE: The Threshold values were determined manually by checking the predictions for the frame_count = [3, 6, 9, 12, 15] and proportion = [5%, 10%, 15%]

schwellenwert_frame_nr_human_attributes = 9
schwellenwert_frame_nr_detectron_2 = 6
schwellenwert_proportion_human_attributes = 0.05
schwellenwert_proportion_detectron_2 = 0

for year in os.listdir(output_folder_path):
  print("Output folder path", output_folder_path)
  print(f"Processing year {year}")
  ad_folder_path = os.path.join(output_folder_path, year)
  print("AD folder path", ad_folder_path)
  for ad in os.listdir(ad_folder_path):
      if ad.endswith(".xlsx"):
          input_file_path = os.path.join(ad_folder_path, ad)
          try:
            generate_summary(input_file_path)
            generate_summary_gender_ethnicity(input_file_path)
          except:
            print(f"Summary for ad {ad} couldn't get created")
            continue